In [3]:
import time

In [6]:
"""
    N-body simulation.
"""
import time

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)

def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m[0] * mag, m[1] * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1_b, v2_b = compute_b([m2, m1], dt, dx, dy, dz)
    v1[0] -= dx * v1_b
    v1[1] -= dy * v1_b
    v1[2] -= dz * v1_b

    v2[0] += dx * v2_b
    v2[1] += dy * v2_b
    v2[2] += dz * v2_b

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                seenit.append(body1)

    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)

def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)

    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.

    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m

    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        start = time.time()
        #report_energy()
        for _ in range(iterations):
            advance(0.01)
        print(report_energy())
        end = time.time()
        print("time ", end - start)

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



-0.16908926275527025
time  0.42011094093322754
-0.1690524049239042
time  0.4315187931060791
-0.169014744045289
time  0.4215991497039795
-0.16902307738079617
time  0.4215984344482422
-0.16907985939165196
time  0.41812729835510254
-0.16908378513401653
time  0.425567626953125
-0.16904612370292355
time  0.43995118141174316
-0.16901416322874255
time  0.4215989112854004
-0.16902778605966304
time  0.42606329917907715
-0.1690837125696274
time  0.42209482192993164


KeyboardInterrupt: 

In [7]:
"""
    N-body simulation.
"""
import time

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)

def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1_b = compute_b(m2, dt, dx, dy, dz)
    v1[0] -= dx * v1_b
    v1[1] -= dy * v1_b
    v1[2] -= dz * v1_b

    v2_b = compute_b(m1, dt, dx, dy, dz)
    v2[0] += dx * v2_b
    v2[1] += dy * v2_b
    v2[2] += dz * v2_b

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                seenit.append(body1)

    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)

def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)

    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.

    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m

    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        start = time.time()
        #report_energy()
        for _ in range(iterations):
            advance(0.01)
        print(report_energy())
        end = time.time()
        print("time ", end - start)

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



-0.16908926275527025
time  0.4860806465148926
-0.1690524049239042
time  0.48508620262145996
-0.169014744045289
time  0.5014550685882568
-0.16902307738079617
time  0.4796304702758789
-0.16907985939165196
time  0.48707127571105957


KeyboardInterrupt: 

In [5]:
a = 3
%timeit list(map(lambda x, y : x - y * a, [10, 9, 8], [1, 2, 3]))

775 ns ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
%%timeit
x = 10 - 3
y = 9 - 2 * 3
z = 8 - 3 * 3

27.5 ns ± 0.518 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [50]:
%timeit -n 10 -r 3 nbody(10, 'sun', 20000)

-0.16956118264377615
time:  0.3057699203491211
-0.1695385468682378
time:  0.30336666107177734
-0.1695189548541446
time:  0.30752015113830566
-0.1695039997413695
time:  0.31743955612182617
-0.16949260597223692
time:  0.3109917640686035
-0.16949265644623607
time:  0.3179354667663574
-0.16951359950772554
time:  0.31743955612182617
-0.1695496271303851
time:  0.311983585357666
-0.1695749762535969
time:  0.3243839740753174
-0.16957686627273363
time:  0.3085498809814453
-0.1694053972072186
time:  0.3058900833129883
-0.16938031957033076
time:  0.3253755569458008
-0.16935113905224383
time:  0.30652832984924316
-0.16932952788732106
time:  0.30454349517822266
-0.16934151083624788
time:  0.3060314655303955
-0.1693899799886623
time:  0.3109917640686035
-0.16942247597292326
time:  0.3080151081085205
-0.16941810749846897
time:  0.3129756450653076
-0.16939584024984417
time:  0.30950403213500977
-0.16936702395426906
time:  0.303483247756958
-0.16942749038260155
time:  0.3035881519317627
-0.169413740759

-0.17036417069764656
time:  0.3169436454772949
-0.1703173266822375
time:  0.32190418243408203
-0.17035112750530565
time:  0.3283510208129883
-0.17036865463385706
time:  0.32789015769958496
-0.17033745032030623
time:  0.32831621170043945
-0.1703147519181412
time:  0.31743931770324707
-0.17036065402857975
time:  0.32884740829467773
-0.17035741915098154
time:  0.34471964836120605
-0.17031791536736107
time:  0.31446385383605957
-0.17032573964393138
time:  0.32289600372314453
-0.17035494690842126
time:  0.3139667510986328
-0.17035754260415448
time:  0.3209116458892822
-0.1703092100605608
time:  0.31446361541748047
-0.17033514822352433
time:  0.3129754066467285
-0.17036048389079628
time:  0.3189280033111572
-0.17071604868423357
time:  0.32686305046081543
-0.1707338377343776
time:  0.3179354667663574
-0.17073118269736628
time:  0.3204152584075928
-0.17072027176573867
time:  0.3184316158294678
-0.1707448927046267
time:  0.31148767471313477
-0.17071264082386953
time:  0.31247949600219727
-0.170

In [43]:
advanced_001 = advance
%timeit [advanced_001(0.01) for _ in range(1000)]

16.7 ms ± 361 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
%timeit [advance(0.01) for _ in range(1000)]

15.8 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
re = report_energy
%timeit [re() for _ in range(1000)]

10.4 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [47]:
%timeit [report_energy() for _ in range(1000)]

10.3 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
                e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
                seenit.append(body1) 
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

In [34]:
%timeit -n 10000 report_energy()

9.91 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [39]:
def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
                
                mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))
                v1_b = m2 * mag
                v1[0] -= dx * v1_b
                v1[1] -= dy * v1_b
                v1[2] -= dz * v1_b

                v2_b = m1 * mag
                v2[0] += dx * v2_b
                v2[1] += dy * v2_b
                v2[2] += dz * v2_b
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        r[0] += dt * vx
        r[1] += dt * vy
        r[2] += dt * vz

In [41]:
%timeit -n 1000 advance(0.01)

15.3 µs ± 600 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
